### Selecting an Ideal site for opening a new rock-climbing gym in Denver, CO.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup 
import requests  
import seaborn as sns

In [2]:
#url =  'statistical_neighborhoods'
df = pd.read_csv('statistical_neighborhoods.csv')

df#.head()

,Borough,Neighborhood,Latitude,Longitude
0,River North,Auraria,39.7506,-105.0075
1,Capitol Hill,Cherry Creek,39.7195,-104.9481
2,Capitol Hill,Congress Park,39.7329,-104.9494
3,Lakeside,West Highland,39.7647,-105.0407
4,Lakeside,Sloan Lake,39.7486,-105.0473
5,Lakeside,Berkeley,39.7766,-105.0397
6,Lakeside,Chaffee Park,39.7872,-105.0146
7,Lakeside,Highland,39.7613,-105.0149
8,Englewood,Wellshire,39.6627,-104.9505
9,Glendale,Montclair,39.7326,-104.9138


In [3]:
df[["Latitude", "Longitude"]] = df[["Latitude", "Longitude"]].astype("float")

In [4]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,River North,Auraria,39.7506,-105.0075
1,Capitol Hill,Cherry Creek,39.7195,-104.9481
2,Capitol Hill,Congress Park,39.7329,-104.9494
3,Lakeside,West Highland,39.7647,-105.0407
4,Lakeside,Sloan Lake,39.7486,-105.0473


In [5]:
address = 'Denver'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver are 39.7392364, -104.9848623.


In [6]:
# create map using latitude and longitude values
map_denver = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denver)  
    
map_denver

In [7]:
CLIENT_ID = 'LHZARKWYYEMMNULRDWIRLIVOF5RICOU5HZSXOCOLCLTWA3X3' # your Foursquare ID
CLIENT_SECRET = 'QGAASYMBSDJFOK5VX4IM4C1M22A43FJYSRUVQC1VU2234BLI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LHZARKWYYEMMNULRDWIRLIVOF5RICOU5HZSXOCOLCLTWA3X3
CLIENT_SECRET:QGAASYMBSDJFOK5VX4IM4C1M22A43FJYSRUVQC1VU2234BLI


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# type your answer here

denver_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Auraria
Cherry Creek
Congress Park
West Highland
Sloan Lake
Berkeley
Chaffee Park
Highland
Wellshire
Montclair
South Park Hill
University Park
Hampden
Indian Creek
University Hills
Harvey Park
Sun Valley
Valverde
Villa Park
Five Points
Bear Valley
Virginia Village
Windsor
Jefferson Park
Whittier
Westwood
Sunnyside


In [10]:
print(denver_venues.shape)
denver_venues.head()

(1285, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Auraria,39.7506,-105.0075,Ball Arena,39.748700,-105.007600,Basketball Stadium
1,Auraria,39.7506,-105.0075,Orangetheory Fitness,39.750981,-105.003716,Gym
2,Auraria,39.7506,-105.0075,Coohills,39.750764,-105.003832,French Restaurant
3,Auraria,39.7506,-105.0075,Confluence Park,39.754487,-105.008569,Park
4,Auraria,39.7506,-105.0075,Kimpton Hotel Born,39.752554,-105.002370,Hotel


In [11]:
denver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Auraria,100,100,100,100,100,100
Bear Valley,4,4,4,4,4,4
Berkeley,95,95,95,95,95,95
Chaffee Park,31,31,31,31,31,31
Cherry Creek,100,100,100,100,100,100
Congress Park,42,42,42,42,42,42
Five Points,100,100,100,100,100,100
Hampden,34,34,34,34,34,34
Harvey Park,7,7,7,7,7,7


In [12]:
print('There are {} uniques categories.'.format(len(denver_venues['Venue Category'].unique())))

There are 245 uniques categories.


In [13]:
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")
denver_onehot['Neighborhood'] = denver_venues['Neighborhood'] 
fixed_columns = [denver_onehot.columns[-1]] + list(denver_onehot.columns[:-1])
denver_onehot = denver_onehot[fixed_columns]
denver_grouped = denver_onehot.groupby('Neighborhood').mean().reset_index()
denver_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Car Wash,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Ski Shop,Smoke Shop,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Taxi,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trade School,Trail,Train Station,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Auraria,0.0,0.0,0.0,0.01,0.050000,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.000000,0.00,0.020000,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.01,0.00,0.0,0.01,0.0,0.000000,0.000000,0.010000,0.030000,0.00,0.01,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.0,0.00,0.01,0.0,0.01,0.010000,0.020000,0.050000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.01,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.000000,

In [14]:
num_top_venues = 5

for hood in denver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = denver_grouped[denver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Auraria----
                 venue  freq
0  American Restaurant  0.05
1          Coffee Shop  0.05
2                Hotel  0.05
3   Mexican Restaurant  0.03
4     Sushi Restaurant  0.03


----Bear Valley----
         venue  freq
0         Park  0.50
1  Video Store  0.25
2         Pool  0.25
3  Music Store  0.00
4       Museum  0.00


----Berkeley----
            venue  freq
0     Coffee Shop  0.06
1         Brewery  0.05
2            Park  0.04
3     Pizza Place  0.04
4  Breakfast Spot  0.04


----Chaffee Park----
                  venue  freq
0  Fast Food Restaurant  0.16
1     Convenience Store  0.10
2          Intersection  0.06
3         Grocery Store  0.06
4                  Food  0.06


----Cherry Creek----
                 venue  freq
0       Cosmetics Shop  0.06
1  American Restaurant  0.05
2       Clothing Store  0.05
3          Coffee Shop  0.05
4   Italian Restaurant  0.04


----Congress Park----
                venue  freq
0                 Bar  0.07
1  Italian Restaura

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = denver_grouped['Neighborhood']

for ind in np.arange(denver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Auraria,American Restaurant,Coffee Shop,Hotel,Mexican Restaurant,Sushi Restaurant,Brewery,Seafood Restaurant,Restaurant,Park,New American Restaurant
1,Bear Valley,Park,Video Store,Pool,Music Store,Museum,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Zoo Exhibit
2,Berkeley,Coffee Shop,Brewery,Park,Pizza Place,Breakfast Spot,Italian Restaurant,American Restaurant,Bar,Mexican Restaurant,Sushi Restaurant
3,Chaffee Park,Fast Food Restaurant,Convenience Store,Intersection,Grocery Store,Food,Park,Discount Store,Hobby Shop,Pharmacy,Clothing Store
4,Cherry Creek,Cosmetics Shop,American Restaurant,Clothing Store,Coffee Shop,Italian Restaurant,Mexican Restaurant,Yoga Studio,Burger Joint,Furniture / Home Store,Restaurant


In [17]:
# set number of clusters
kclusters = 5

denver_grouped_clustering = denver_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 4, 1, 4, 4, 4, 4, 3, 4])

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = df

denver_merged = denver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

denver_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,River North,Auraria,39.7506,-105.0075,4,American Restaurant,Coffee Shop,Hotel,Mexican Restaurant,Sushi Restaurant,Brewery,Seafood Restaurant,Restaurant,Park,New American Restaurant
1,Capitol Hill,Cherry Creek,39.7195,-104.9481,4,Cosmetics Shop,American Restaurant,Clothing Store,Coffee Shop,Italian Restaurant,Mexican Restaurant,Yoga Studio,Burger Joint,Furniture / Home Store,Restaurant
2,Capitol Hill,Congress Park,39.7329,-104.9494,4,Bar,Italian Restaurant,Coffee Shop,Yoga Studio,Grocery Store,Pizza Place,Café,Breakfast Spot,Botanical Garden,Cosmetics Shop
3,Lakeside,West Highland,39.7647,-105.0407,4,Coffee Shop,Pizza Place,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Brewery,Bar,Liquor Store,Italian Restaurant,Ice Cream Shop
4,Lakeside,Sloan Lake,39.7486,-105.0473,4,Coffee Shop,Fast Food Restaurant,Sandwich Place,Bar,Ice Cream Shop,Marijuana Dispensary,Mobile Phone Shop,Spa,Ramen Restaurant,Convenience Store


In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighborhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
clus1=denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Windsor,Gym,Golf Course,American Restaurant,Lake,Lawyer,Grocery Store,Restaurant,Zoo Exhibit,Movie Theater,Music Store


In [21]:
clus2=denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Chaffee Park,Fast Food Restaurant,Convenience Store,Intersection,Grocery Store,Food,Park,Discount Store,Hobby Shop,Pharmacy,Clothing Store
8,Wellshire,Golf Course,Pharmacy,Church,Mexican Restaurant,Bus Station,Bus Stop,Grocery Store,Gym / Fitness Center,Playground,Arcade
9,Montclair,Park,Mexican Restaurant,Grocery Store,Chinese Restaurant,Hardware Store,Asian Restaurant,Coffee Shop,Pharmacy,Fast Food Restaurant,Flower Shop
17,Valverde,Mexican Restaurant,Marijuana Dispensary,Fast Food Restaurant,Convenience Store,Dim Sum Restaurant,Pharmacy,Gym / Fitness Center,Liquor Store,Vietnamese Restaurant,Print Shop
21,Virginia Village,Coffee Shop,Construction & Landscaping,Gym,Playground,Discount Store,Bistro,Liquor Store,Convenience Store,Irish Pub,Israeli Restaurant
25,Westwood,Mexican Restaurant,Food,Hobby Shop,Coffee Shop,Liquor Store,Locksmith,Marijuana Dispensary,Museum,Nightclub,Diner
26,Sunnyside,Coffee Shop,Marijuana Dispensary,Mexican Restaurant,Sandwich Place,Recreation Center,Food,Beer Store,Pizza Place,Breakfast Spot,Brewery


In [22]:
clus3=denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Bear Valley,Park,Video Store,Pool,Music Store,Museum,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Zoo Exhibit


In [23]:
clus4=denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Harvey Park,Construction & Landscaping,Recreation Center,Lake,Park,Home Service,Pet Store,Pharmacy,Massage Studio,Medical Center,Mediterranean Restaurant


In [24]:
clus5=denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]
clus5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Auraria,American Restaurant,Coffee Shop,Hotel,Mexican Restaurant,Sushi Restaurant,Brewery,Seafood Restaurant,Restaurant,Park,New American Restaurant
1,Cherry Creek,Cosmetics Shop,American Restaurant,Clothing Store,Coffee Shop,Italian Restaurant,Mexican Restaurant,Yoga Studio,Burger Joint,Furniture / Home Store,Restaurant
2,Congress Park,Bar,Italian Restaurant,Coffee Shop,Yoga Studio,Grocery Store,Pizza Place,Café,Breakfast Spot,Botanical Garden,Cosmetics Shop
3,West Highland,Coffee Shop,Pizza Place,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Brewery,Bar,Liquor Store,Italian Restaurant,Ice Cream Shop
4,Sloan Lake,Coffee Shop,Fast Food Restaurant,Sandwich Place,Bar,Ice Cream Shop,Marijuana Dispensary,Mobile Phone Shop,Spa,Ramen Restaurant,Convenience Store
5,Berkeley,Coffee Shop,Brewery,Park,Pizza Place,Breakfast Spot,Italian Restaurant,American Restaurant,Bar,Mexican Restaurant,Sushi Restaurant
7,Highland,Coffee Shop,Pizza Place,Italian Restaurant,American Restaurant,Brewery,Cocktail Bar,Yoga Studio,Café,Sushi Restaurant,Mexican Restaurant
10,South Park Hill,Coffee Shop,Ethiopian Restaurant,Marijuana Dispensary,Grocery Store,Pizza Place,American Restaurant,Gym / Fitness Center,Furniture / Home Store,Fast Food Restaurant,Convenience Store
11,University Park,Sandwich Place,Organic Grocery,Japanese Restaurant,Salad Place,Bar,Sushi Restaurant,Bakery,Mediterranean Restaurant,Greek Restaurant,Pizza Place
12,Hampden,Pizza Place,Mexican Restaurant,Park,Japanese Restaurant,Bar,Deli / Bodega,Seafood Restaurant,Sandwich Place,Salon / Barbershop,Chinese Restaurant
